In [241]:
#Import the packages we will need 
#Requests to interact with the api, pandas for working with the data, numpy for data calcs

import requests
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import json

In [242]:
def get_recent_gameweek_id():
    """
    Get's the most recent gameweek's ID.
    """

    data = requests.get(url)
    data = json.loads(data.content)

    gameweeks = data['events']
    
    now = datetime.utcnow()
    for gameweek in gameweeks:
        next_deadline_date = datetime.strptime(gameweek['deadline_time'], '%Y-%m-%dT%H:%M:%SZ')
        if next_deadline_date > now:
            return gameweek['id'] 


if __name__ == '__main__':
     GW = float(get_recent_gameweek_id())
    
GW
    

31.0

In [243]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

r = requests.get(url)

json = r.json()

#This lets us examine the JSON objext so we know what to do with it


In [244]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
elements_stats_df = pd.DataFrame(json['element_stats'])
teams_df = pd.DataFrame(json['teams'])
events_df = pd.DataFrame(json['events'])
phases_df = pd.DataFrame(json['phases'])


teams_df.rename(columns={'strength_overall_home': 'strength_overall_away', 'strength_overall_away': 'strength_overall_home'}, inplace=True)



In [245]:
fig = go.Figure([go.Bar(name="home", x=teams_df["name"], y= teams_df["strength_overall_home"],offsetgroup=0,),
                 go.Bar(name ="away", x=teams_df["name"], y= teams_df["strength_overall_away"],offsetgroup=1,),],layout=go.Layout(
        title="Overall strength"))              

fig2 = go.Figure([go.Bar(name="home", x=teams_df["name"], y= teams_df["strength_attack_home"],offsetgroup=0,),
                 go.Bar(name ="away", x=teams_df["name"], y= teams_df["strength_attack_away"],offsetgroup=1,),],layout=go.Layout(
        title="Attacking strength"))

fig3 = go.Figure([go.Bar(name="home", x=teams_df["name"], y= teams_df["strength_defence_home"],offsetgroup=0,),
                 go.Bar(name ="away", x=teams_df["name"], y= teams_df["strength_defence_away"],offsetgroup=1,),],layout=go.Layout(
        title="Defensive strength"))

#fig.show()

#fig2.show()

#fig3.show()

In [246]:

slim_elements_df = elements_df[['first_name','second_name','team','element_type','minutes','selected_by_percent',
                                'now_cost','minutes','transfers_in','transfers_out','value_season','value_form','form','total_points','points_per_game',
                               'threat','influence','creativity','news']].copy()


slim_elements_df.head(5)

,first_name,second_name,team,element_type,minutes,selected_by_percent,now_cost,minutes,transfers_in,transfers_out,value_season,value_form,form,total_points,points_per_game,threat,influence,creativity,news
0,Mesut,Özil,1,3,0,0.5,67,0,3441,54723,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,Not included in Arsenal's 25-man Premier Leagu...
1,Sokratis,Papastathopoulos,1,2,0,0.1,48,0,10266,19117,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,Left the club by mutual consent on 20/1
2,David,Luiz Moreira Marinho,1,2,1344,0.8,54,1344,76497,123486,7.4,0.2,1.0,40,2.1,123.0,244.6,45.2,Knee injury - Unknown return date
3,Pierre-Emerick,Aubameyang,1,3,2007,9.0,114,2007,1131009,3620784,9.5,0.1,1.3,108,4.3,731.0,428.2,292.4,
4,Cédric,Soares,1,2,744,0.3,46,744,35648,53674,6.1,0.2,0.7,28,2.8,66.0,110.8,114.8,


In [247]:
slim_elements_df.loc[:,'position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)
slim_elements_df.head()

,first_name,second_name,team,element_type,minutes,selected_by_percent,now_cost,minutes,transfers_in,transfers_out,value_season,value_form,form,total_points,points_per_game,threat,influence,creativity,news,position
0,Mesut,Özil,1,3,0,0.5,67,0,3441,54723,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,Not included in Arsenal's 25-man Premier Leagu...,Midfielder
1,Sokratis,Papastathopoulos,1,2,0,0.1,48,0,10266,19117,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,Left the club by mutual consent on 20/1,Defender
2,David,Luiz Moreira Marinho,1,2,1344,0.8,54,1344,76497,123486,7.4,0.2,1.0,40,2.1,123.0,244.6,45.2,Knee injury - Unknown return date,Defender
3,Pierre-Emerick,Aubameyang,1,3,2007,9.0,114,2007,1131009,3620784,9.5,0.1,1.3,108,4.3,731.0,428.2,292.4,,Midfielder
4,Cédric,Soares,1,2,744,0.3,46,744,35648,53674,6.1,0.2,0.7,28,2.8,66.0,110.8,114.8,,Defender


In [248]:
slim_elements_df['team'] = slim_elements_df.team.map(teams_df.set_index('id').name)

slim_elements_df['value'] = slim_elements_df.value_season.astype(float)

#slim_elements_df['games'] = slim_elements_df.games.astype(float)

slim_elements_df['total_points'] = slim_elements_df.total_points.astype(float)

slim_elements_df.sort_values('value',ascending=False).head(10)

,first_name,second_name,team,element_type,minutes,selected_by_percent,now_cost,minutes,transfers_in,transfers_out,...,value_form,form,total_points,points_per_game,threat,influence,creativity,news,position,value
33,Emiliano,Martínez,Aston Villa,1,2610,40.5,54,2610,3859323,1386101,...,0.5,2.7,157.0,5.4,0.0,784.0,20.0,,Goalkeeper,29.1
324,Illan,Meslier,Leeds,1,2610,6.8,47,2610,979940,588709,...,1.1,5.3,127.0,4.4,0.0,852.4,10.0,,Goalkeeper,27.0
311,Stuart,Dallas,Leeds,2,2690,23.6,51,2690,2972969,1457063,...,0.6,3.3,124.0,4.1,386.0,593.0,357.9,,Defender,24.3
261,Alphonse,Areola,Fulham,1,2700,5.2,46,2700,573363,301915,...,0.3,1.3,111.0,3.7,0.0,723.0,0.0,,Goalkeeper,24.1
313,Patrick,Bamford,Leeds,4,2499,46.1,66,2499,7194011,4339000,...,0.7,4.7,158.0,5.3,1252.0,699.0,293.2,,Forward,23.9
377,Ilkay,Gündogan,Man City,3,1739,31.7,60,1739,4081340,1629291,...,0.4,2.7,142.0,6.2,609.0,656.6,544.0,,Midfielder,23.7
124,Nick,Pope,Burnley,1,2610,12.8,55,2610,1033036,1221560,...,0.4,2.0,130.0,4.5,0.0,786.6,20.0,,Goalkeeper,23.6
379,John,Stones,Man City,2,1620,21.9,52,1620,2673109,1026394,...,0.9,4.7,122.0,6.8,164.0,454.8,56.7,,Defender,23.5
635,Tomas,Soucek,West Ham,3,2700,13.3,53,2700,2817050,2169461,...,0.5,2.7,123.0,4.1,778.0,713.6,193.0,,Midfielder,23.2
48,Matt,Targett,Aston Villa,2,2594,14.1,51,2594,1700043,683837,...,0.4,2.0,118.0,4.1,96.0,489.2,388.3,,Defender,23.1


In [249]:

# Function for creating cell-values
def unavailable(row):
    if row['news'] != '':
        return True
    else:
        return False
    


# Using function to create new column:
slim_elements_df['unavailable'] = slim_elements_df.apply(lambda row: unavailable(row), axis=1)

slim_elements_df.head()

,first_name,second_name,team,element_type,minutes,selected_by_percent,now_cost,minutes,transfers_in,transfers_out,...,form,total_points,points_per_game,threat,influence,creativity,news,position,value,unavailable
0,Mesut,Özil,Arsenal,3,0,0.5,67,0,3441,54723,...,0.0,0.0,0.0,0.0,0.0,0.0,Not included in Arsenal's 25-man Premier Leagu...,Midfielder,0.0,True
1,Sokratis,Papastathopoulos,Arsenal,2,0,0.1,48,0,10266,19117,...,0.0,0.0,0.0,0.0,0.0,0.0,Left the club by mutual consent on 20/1,Defender,0.0,True
2,David,Luiz Moreira Marinho,Arsenal,2,1344,0.8,54,1344,76497,123486,...,1.0,40.0,2.1,123.0,244.6,45.2,Knee injury - Unknown return date,Defender,7.4,True
3,Pierre-Emerick,Aubameyang,Arsenal,3,2007,9.0,114,2007,1131009,3620784,...,1.3,108.0,4.3,731.0,428.2,292.4,,Midfielder,9.5,False
4,Cédric,Soares,Arsenal,2,744,0.3,46,744,35648,53674,...,0.7,28.0,2.8,66.0,110.8,114.8,,Defender,6.1,False


In [250]:


slim_elements_df['strength_overall_home'] = slim_elements_df.team.map(teams_df.set_index('name')['strength_overall_home'].to_dict())

slim_elements_df['strength_overall_away'] = slim_elements_df.team.map(teams_df.set_index('name')['strength_overall_away'].to_dict())

slim_elements_df.head()

,first_name,second_name,team,element_type,minutes,selected_by_percent,now_cost,minutes,transfers_in,transfers_out,...,points_per_game,threat,influence,creativity,news,position,value,unavailable,strength_overall_home,strength_overall_away
0,Mesut,Özil,Arsenal,3,0,0.5,67,0,3441,54723,...,0.0,0.0,0.0,0.0,Not included in Arsenal's 25-man Premier Leagu...,Midfielder,0.0,True,1250,1200
1,Sokratis,Papastathopoulos,Arsenal,2,0,0.1,48,0,10266,19117,...,0.0,0.0,0.0,0.0,Left the club by mutual consent on 20/1,Defender,0.0,True,1250,1200
2,David,Luiz Moreira Marinho,Arsenal,2,1344,0.8,54,1344,76497,123486,...,2.1,123.0,244.6,45.2,Knee injury - Unknown return date,Defender,7.4,True,1250,1200
3,Pierre-Emerick,Aubameyang,Arsenal,3,2007,9.0,114,2007,1131009,3620784,...,4.3,731.0,428.2,292.4,,Midfielder,9.5,False,1250,1200
4,Cédric,Soares,Arsenal,2,744,0.3,46,744,35648,53674,...,2.8,66.0,110.8,114.8,,Defender,6.1,False,1250,1200


In [251]:
grouped_se_df = slim_elements_df.groupby(slim_elements_df.position)

df_gk = grouped_se_df.get_group("Goalkeeper").copy()
df_df = grouped_se_df.get_group("Defender").copy()
df_mfd = grouped_se_df.get_group("Midfielder").copy()
df_fwd = grouped_se_df.get_group("Forward").copy()

df_gk.sort_values(['value'],inplace =True)
df_df.sort_values(['value'],inplace =True)
df_mfd.sort_values(['value'],inplace =True)
df_fwd.sort_values(['value'],inplace =True)


fig_ovr_val = px.scatter(slim_elements_df, x="now_cost", y="total_points", hover_data=['second_name'])


fig_gk = px.bar(df_gk, x = 'second_name', y = 'value',hover_data = ['team'])
fig_df = px.bar(df_df, x = 'second_name', y = 'value',hover_data = ['team'])
fig_mfd = px.bar(df_mfd, x = 'second_name', y = 'value',hover_data = ['team'])
fig_fwd = px.bar(df_fwd, x = 'second_name', y = 'value',hover_data = ['team'])

fig_gk_tp = px.scatter(df_gk, x="second_name", y="total_points", hover_data=['now_cost'])
fig_df_tp = px.scatter(df_df, x="second_name", y="total_points", hover_data=['now_cost'])
fig_mfd_tp = px.scatter(df_mfd, x="second_name", y="total_points", hover_data=['now_cost'])
fig_fwd_tp = px.scatter(df_fwd, x="second_name", y="total_points", hover_data=['now_cost'])

fig_gk.add_trace(fig_gk_tp.data[0])
fig_df.add_trace(fig_df_tp.data[0])
fig_mfd.add_trace(fig_mfd_tp.data[0])
fig_fwd.add_trace(fig_fwd_tp.data[0])

fig_ovr_val.show()

fig_gk.show()
fig_df.show()
fig_mfd.show()
fig_fwd.show()

In [252]:
slim_elements_df_stars = slim_elements_df.copy()

slim_elements_df_stars.sort_values(['total_points'],inplace = True, ascending = False)

In [253]:
slim_elements_df_stars.head(5)

,first_name,second_name,team,element_type,minutes,selected_by_percent,now_cost,minutes,transfers_in,transfers_out,...,points_per_game,threat,influence,creativity,news,position,value,unavailable,strength_overall_home,strength_overall_away
418,Bruno Miguel,Borges Fernandes,Man Utd,3,2555,58.0,116,2555,5833564,2720043,...,7.1,993.0,1150.4,1255.2,,Midfielder,18.4,False,1320,1250
551,Harry,Kane,Spurs,4,2453,48.5,117,2453,7242543,4893714,...,7.3,1250.0,1106.4,519.0,,Forward,17.4,False,1250,1190
351,Mohamed,Salah,Liverpool,3,2432,30.5,124,2432,5281821,5544909,...,6.3,1408.0,838.0,614.8,,Midfielder,14.7,False,1210,1250
553,Heung-Min,Son,Spurs,3,2399,42.2,94,2399,7533411,6663188,...,6.2,775.0,815.0,742.3,,Midfielder,19.1,False,1250,1190
313,Patrick,Bamford,Leeds,4,2499,46.1,66,2499,7194011,4339000,...,5.3,1252.0,699.0,293.2,,Forward,23.9,False,1170,1160


In [254]:
slim_elements_df_roi = slim_elements_df.copy()

slim_elements_df_roi.sort_values(['value'], inplace = True, ascending = False)

slim_elements_df_roi.head()

,first_name,second_name,team,element_type,minutes,selected_by_percent,now_cost,minutes,transfers_in,transfers_out,...,points_per_game,threat,influence,creativity,news,position,value,unavailable,strength_overall_home,strength_overall_away
33,Emiliano,Martínez,Aston Villa,1,2610,40.5,54,2610,3859323,1386101,...,5.4,0.0,784.0,20.0,,Goalkeeper,29.1,False,1160,1100
324,Illan,Meslier,Leeds,1,2610,6.8,47,2610,979940,588709,...,4.4,0.0,852.4,10.0,,Goalkeeper,27.0,False,1170,1160
311,Stuart,Dallas,Leeds,2,2690,23.6,51,2690,2972969,1457063,...,4.1,386.0,593.0,357.9,,Defender,24.3,False,1170,1160
261,Alphonse,Areola,Fulham,1,2700,5.2,46,2700,573363,301915,...,3.7,0.0,723.0,0.0,,Goalkeeper,24.1,False,1100,1090
313,Patrick,Bamford,Leeds,4,2499,46.1,66,2499,7194011,4339000,...,5.3,1252.0,699.0,293.2,,Forward,23.9,False,1170,1160


In [255]:
#Theoretical best team 


def team_selecter():
    
    budget = 1000

    position_dict = {"Goalkeeper":2,"Defender":5,"Midfielder": 5, "Forward": 3}

    team_dict = {'West Brom':3,'Sheffield Utd':3,'Burnley':3,'Southampton':3,
             'Fulham':3,'Newcastle':3,'Brighton':3,'Crystal Palace':3,
            'Leeds':3,'Wolves':3,'Aston Villa':3,'Arsenal':3,
             'Spurs':3,'West Ham':3,'Leicester':3,'Everton':3,
            'Liverpool':3,'Chelsea':3,'Man Utd':3,'Man City':3}

    roi_team = [] #form an empty tuple for the team
    
    total_points = 0
    
    top_performer_limit = 3
    
    # Choosing 3 top performers from the "top players"-dataframe
    for idx, row in slim_elements_df_stars.iterrows():
        if budget >= row.now_cost and len(roi_team) < top_performer_limit and row.unavailable == False and position_dict[row.position] != 0 and team_dict[row.team] != 0:
            roi_team.append(row.second_name)
            budget -= row.now_cost #Deducting cost from budget
            position_dict[row.position] -= 1 # Deducting position from position dictionary
            team_dict[row.team] -= 1 # Deducting player from team dictionary
            total_points += row.total_points # adding to point score
            print("Player choosen from 'top players' " + str(row.second_name))
            
                   
        else:
            for idx, row in slim_elements_df_roi.iterrows():
                if row.second_name not in roi_team and budget >= row.now_cost and row.unavailable == False and position_dict[row.position] != 0 and team_dict[row.team] != 0:
                    roi_team.append(row.second_name)
                    budget -= row.now_cost
                    position_dict[row.position] -= 1 # Deducting position from position dictionary
                    team_dict[row.team] -= 1 # Deducting player from team dictionary
                    total_points += row.total_points # adding to point score
    
                   
    print("\nTeam chosen: " + str(roi_team))
    print("Remaining budget: " + str((budget/10)) + "M.")
    print("Total points from choosen team: " + str(total_points) + ".")
    print(position_dict)

In [271]:
filename = "epl-2020-GMTStandardTime.csv"

fixtures = pd.read_csv("/Users/jamesdawson/Desktop/Data analysis /Python/" + filename) 

fixtures_filtered = fixtures.loc[fixtures['Round Number'] == 31].copy()

fixtures_filtered.head()


,Round Number,Date,Location,Home Team,Away Team,Result
301,31,09/04/2021 20:00,Craven Cottage,Fulham,Wolves,NaN
302,31,10/04/2021 12:30,Etihad Stadium,Man City,Leeds,NaN
303,31,10/04/2021 15:00,Anfield,Liverpool,Aston Villa,NaN
304,31,10/04/2021 17:30,Selhurst Park,Crystal Palace,Chelsea,NaN
305,31,11/04/2021 12:00,Turf Moor,Burnley,Newcastle,NaN


In [279]:
fixtures_flipped = fixtures.copy()

fixtures_flipped.rename(columns={'Home Team': 'Away Team', 'Away Team': 'Home Team'}, inplace=True)

fixtures_flipped_filtered = fixtures_flipped.loc[fixtures_flipped['Round Number'] == 31].copy()


fixtures_flipped_filtered.head()


combined_fixtures = fixtures_filtered.append(fixtures_flipped_filtered, ignore_index=True)

combined_fixtures.head(20)

,Round Number,Date,Location,Home Team,Away Team,Result
0,31,09/04/2021 20:00,Craven Cottage,Fulham,Wolves,NaN
1,31,10/04/2021 12:30,Etihad Stadium,Man City,Leeds,NaN
2,31,10/04/2021 15:00,Anfield,Liverpool,Aston Villa,NaN
3,31,10/04/2021 17:30,Selhurst Park,Crystal Palace,Chelsea,NaN
4,31,11/04/2021 12:00,Turf Moor,Burnley,Newcastle,NaN
5,31,11/04/2021 14:05,London Stadium,West Ham,Leicester,NaN
6,31,11/04/2021 16:30,Tottenham Hotspur Stadium,Spurs,Man Utd,NaN
7,31,11/04/2021 19:00,Bramall Lane,Sheffield Utd,Arsenal,NaN
8,31,12/04/2021 18:00,The Hawthorns,West Brom,Southampton,NaN
9,31,12/04/2021 20:15,Amex Stadium,Brighton,Everton,NaN


In [281]:
slim_elements_df['opponent'] = slim_elements_df.team.map(combined_fixtures.set_index('Home Team')['Away Team'].to_dict())

slim_elements_df.head()

,first_name,second_name,team,element_type,minutes,selected_by_percent,now_cost,minutes,transfers_in,transfers_out,...,threat,influence,creativity,news,position,value,unavailable,strength_overall_home,strength_overall_away,opponent
0,Mesut,Özil,Arsenal,3,0,0.5,67,0,3441,54723,...,0.0,0.0,0.0,Not included in Arsenal's 25-man Premier Leagu...,Midfielder,0.0,True,1250,1200,Sheffield Utd
1,Sokratis,Papastathopoulos,Arsenal,2,0,0.1,48,0,10266,19117,...,0.0,0.0,0.0,Left the club by mutual consent on 20/1,Defender,0.0,True,1250,1200,Sheffield Utd
2,David,Luiz Moreira Marinho,Arsenal,2,1344,0.8,54,1344,76497,123486,...,123.0,244.6,45.2,Knee injury - Unknown return date,Defender,7.4,True,1250,1200,Sheffield Utd
3,Pierre-Emerick,Aubameyang,Arsenal,3,2007,9.0,114,2007,1131009,3620784,...,731.0,428.2,292.4,,Midfielder,9.5,False,1250,1200,Sheffield Utd
4,Cédric,Soares,Arsenal,2,744,0.3,46,744,35648,53674,...,66.0,110.8,114.8,,Defender,6.1,False,1250,1200,Sheffield Utd


In [258]:
#team_selecter()